## Working with TiTiler-EoPF


#### Requirements
- folium
- httpx

`pip install folium httpx`

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install folium httpx

In [ ]:
import json
import httpx
from folium import Map, TileLayer

%matplotlib inline

In [ ]:
titiler_endpoint = "http://127.0.0.1:8000"

### Conformances

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/conformance",
).json()

print(json.dumps(r, indent=4))

In [ ]:
collection_id = "eopf_geozarr"
item_id = "S2A_MSIL2A_20250704T094051_N0511_R036_T33SWB_20250704T115824"

## Variables

In [ ]:
# Fetch Metadata for all available variables
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/dataset/keys",
    timeout=10,
).json()

print("list of supported variables: ")
print(json.dumps(list(r), indent=4))

### Info

In [ ]:
# Fetch Metadata for all available variables
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/info",
    timeout=10,
).json()

print(json.dumps(r, indent=4))

In [ ]:
# Fetch Metadata for a specific Variable
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/info",
    params={
        "variables": "/measurements/reflectance/r60m:b05",
    },
    timeout=10,
).json()

print(json.dumps(r, indent=4))

## Display tiles

### Single Variable

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params={
        "variables": "/quality/l2a_quicklook/r10m:tci",
    },
    timeout=10,
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=8
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)

m

### Band Combination: False Color Nir/Red/Green

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params=[
        ("variables", "/measurements/reflectance/r10m:b08"),
        ("variables", "/measurements/reflectance/r10m:b04"),
        ("variables", "/measurements/reflectance/r10m:b03"),
        ("rescale", "0,1"),
    ],
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=9
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)

m

### Band Math: NDVI

In [ ]:
red = "/measurements/reflectance/r10m:b04"
nir = "/measurements/reflectance/r10m:b08"

r = httpx.get(
    f"{titiler_endpoint}/collections/{collection_id}/items/{item_id}/WebMercatorQuad/tilejson.json",
    params=[
        ("expression", f"({nir}-{red})/({nir}+{red})"),
        ("rescale", "-1,1"),
        ("colormap_name", "viridis"),
    ],
    timeout=10,
).json()
print(r)
bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=9
)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="ESA EoPF").add_to(m)

m